# Lab | Making predictions with logistic regression

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix, roc_auc_score

In [3]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [4]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

2. Create a query to get the total amount of rentals in June for each film.

In [5]:
query = '''select f.rental_rate, f.length, f.film_id, f.title, count(rental_id) as June from film as f
join inventory as i on f.film_id = i.film_id
join rental as r on i.inventory_id = r.inventory_id
where r.rental_date >= date('2005-06-01')
and r.rental_date < date('2005-07-01')
group by f.film_id;'''

data2=pd.read_sql_query(query, engine)
data2.head()

,rental_rate,length,film_id,title,June
0,2.99,102,356,GIANT TROOPERS,4
1,4.99,161,961,WASH HEAVENLY,5
2,4.99,178,614,NAME DETECTIVE,4
3,4.99,56,369,GOODFELLAS SALUTE,5
4,2.99,106,220,DEER VIRGINIAN,4


3. Do the same with July.

In [6]:
query = '''select f.rating, f.film_id, f.title, count(rental_id) as July from film as f
join inventory as i on f.film_id = i.film_id
join rental as r on i.inventory_id = r.inventory_id
where r.rental_date >= date('2005-07-01')
and r.rental_date < date('2005-08-01')
group by f.film_id;'''

data3 = pd.read_sql_query(query, engine)
data3.head()

,rating,film_id,title,July
0,PG,194,CROW GREASE,7
1,G,376,GRAPES FURY,11
2,PG-13,184,CORE SUIT,9
3,PG,600,MOTIONS DETAILS,5
4,PG,19,AMADEUS HOLY,8


In [7]:
data3.head(100)

,rating,film_id,title,July
0,PG,194,CROW GREASE,7
1,G,376,GRAPES FURY,11
2,PG-13,184,CORE SUIT,9
3,PG,600,MOTIONS DETAILS,5
4,PG,19,AMADEUS HOLY,8
...,...,...,...,...
95,NC-17,451,IGBY MAKER,8
96,PG-13,418,HOBBIT ALIEN,10
97,R,287,ENTRAPMENT SATISFACTION,5
98,PG,906,TRAMP OTHERS,8


4. Create a new column containing (Yes/No) for each film whether or not the number of monthly rentals in July was bigger than in June. Your objective will be to predict this new column.

In [8]:
num_rentals = data2.merge(data3,how='outer',on='film_id')
num_rentals

,rental_rate,length,film_id,title_x,June,rating,title_y,July
0,2.99,102.0,356,GIANT TROOPERS,4.0,R,GIANT TROOPERS,11
1,4.99,161.0,961,WASH HEAVENLY,5.0,R,WASH HEAVENLY,7
2,4.99,178.0,614,NAME DETECTIVE,4.0,PG-13,NAME DETECTIVE,11
3,4.99,56.0,369,GOODFELLAS SALUTE,5.0,PG,GOODFELLAS SALUTE,13
4,2.99,106.0,220,DEER VIRGINIAN,4.0,NC-17,DEER VIRGINIAN,13
...,...,...,...,...,...,...,...,...
953,NaN,NaN,485,NaN,NaN,PG,JERSEY SASSY,3
954,NaN,NaN,939,NaN,NaN,R,VERTIGO NORTHWEST,3
955,NaN,NaN,904,NaN,NaN,R,TRAIN BUNCH,1
956,NaN,NaN,400,NaN,NaN,R,HARDLY ROBBERS,2


In [9]:
num_rentals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958 entries, 0 to 957
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rental_rate  900 non-null    float64
 1   length       900 non-null    float64
 2   film_id      958 non-null    int64  
 3   title_x      900 non-null    object 
 4   June         900 non-null    float64
 5   rating       958 non-null    object 
 6   title_y      958 non-null    object 
 7   July         958 non-null    int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 67.4+ KB


In [10]:
num_rentals['target']=num_rentals['July'] > num_rentals['June']

In [11]:
num_rentals['target'] = num_rentals['target'].replace({True:'Yes',False:'No'})

In [12]:
num_rentals.head()

,rental_rate,length,film_id,title_x,June,rating,title_y,July,target
0,2.99,102.0,356,GIANT TROOPERS,4.0,R,GIANT TROOPERS,11,Yes
1,4.99,161.0,961,WASH HEAVENLY,5.0,R,WASH HEAVENLY,7,Yes
2,4.99,178.0,614,NAME DETECTIVE,4.0,PG-13,NAME DETECTIVE,11,Yes
3,4.99,56.0,369,GOODFELLAS SALUTE,5.0,PG,GOODFELLAS SALUTE,13,Yes
4,2.99,106.0,220,DEER VIRGINIAN,4.0,NC-17,DEER VIRGINIAN,13,Yes


In [13]:
num_rentals['target'].value_counts()

Yes    894
No      64
Name: target, dtype: int64

In [14]:
num_rentals1=num_rentals.dropna() # dropping all NaN values

In [15]:
num_rentals1=num_rentals1.drop(['title_y'],axis=1) # dropping title

In [16]:
num_rentals1=num_rentals1.rename({"title_x": "title"}) #renaming the column

5. Read the data into a Pandas dataframe.

In [18]:
num_rentals1

,rental_rate,length,film_id,title_x,June,rating,July,target
0,2.99,102.0,356,GIANT TROOPERS,4.0,R,11,Yes
1,4.99,161.0,961,WASH HEAVENLY,5.0,R,7,Yes
2,4.99,178.0,614,NAME DETECTIVE,4.0,PG-13,11,Yes
3,4.99,56.0,369,GOODFELLAS SALUTE,5.0,PG,13,Yes
4,2.99,106.0,220,DEER VIRGINIAN,4.0,NC-17,13,Yes
...,...,...,...,...,...,...,...,...
895,0.99,79.0,601,MOULIN WAKE,1.0,PG-13,8,Yes
896,2.99,138.0,519,LIBERTY MAGNIFICENT,1.0,G,3,Yes
897,4.99,153.0,47,BABY HALL,1.0,NC-17,3,Yes
898,4.99,177.0,248,DOZEN LION,1.0,NC-17,2,Yes


In [19]:
num_rentals1=num_rentals1.drop(['June', 'July', 'title_x'],axis=1)

In [20]:
num_rentals1

,rental_rate,length,film_id,rating,target
0,2.99,102.0,356,R,Yes
1,4.99,161.0,961,R,Yes
2,4.99,178.0,614,PG-13,Yes
3,4.99,56.0,369,PG,Yes
4,2.99,106.0,220,NC-17,Yes
...,...,...,...,...,...
895,0.99,79.0,601,PG-13,Yes
896,2.99,138.0,519,G,Yes
897,4.99,153.0,47,NC-17,Yes
898,4.99,177.0,248,NC-17,Yes


6. Analyze extracted features and transform them. You may need to encode some categorical variables or scale numerical variables.

In [21]:
# checking data for null values and datatype of columns
num_rentals1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rental_rate  900 non-null    float64
 1   length       900 non-null    float64
 2   film_id      900 non-null    int64  
 3   rating       900 non-null    object 
 4   target       900 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 42.2+ KB


In [22]:
num_rentals1.isna().sum() # checking missing values

rental_rate    0
length         0
film_id        0
rating         0
target         0
dtype: int64

In [23]:
num_rentals1.duplicated() #checking duplicated values

0      False
1      False
2      False
3      False
4      False
       ...  
895    False
896    False
897    False
898    False
899    False
Length: 900, dtype: bool

In [24]:
num_rentals1['film_id'].duplicated().sum() # checking duplicates on film_id column

0

7. Create a logistic regression model to predict this new column from the cleaned data.

In [25]:
#splitting data
y = num_rentals1['target']
X = num_rentals1.drop (['target'],axis=1)

In [26]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.40, random_state=100, stratify = y)

In [27]:
## dealing with numerical column

In [28]:
X_train_num = X_train.select_dtypes('number')
X_test_num = X_test.select_dtypes('number')

In [29]:
from sklearn.preprocessing import StandardScaler
import numpy as np

In [30]:
transformer = StandardScaler().fit(X_train_num)
X_train_scaled_np = transformer.transform(X_train_num)
X_test_scaled_np = transformer.transform(X_test_num)

X_train_scaled = pd.DataFrame(X_train_scaled_np,columns=X_train_num.columns)
X_test_scaled = pd.DataFrame(X_test_scaled_np,columns =X_test_num.columns)

In [37]:
X_train_scaled

,rental_rate,length,film_id
0,1.217078,1.599174,-1.101022
1,-1.199180,-1.574481,-0.694295
2,-1.199180,-0.418188,-1.387110
3,0.008949,1.722184,0.677546
4,1.217078,-1.500675,0.984315
...,...,...,...
535,1.217078,-1.279257,-0.094546
536,-1.199180,-0.368984,-0.366846
537,1.217078,0.934920,-0.763232
538,-1.199180,-1.525277,1.270403


In [31]:
## dealing with categorical column

In [32]:
from sklearn.preprocessing import OneHotEncoder
X_train_categorical = X_train.select_dtypes('object')
X_test_categorical =  X_test.select_dtypes('object')

In [33]:
X_test_categorical

,rating
622,PG-13
333,R
598,R
423,PG
783,PG
...,...
417,NC-17
696,PG-13
578,PG-13
730,R


In [34]:
encoder = OneHotEncoder() 
encoder.fit(X_train_categorical)

X_train_cat_np = encoder.transform(X_train_categorical).toarray()
X_test_cat_np = encoder.transform(X_test_categorical).toarray()

X_train_cat = pd.DataFrame(X_train_cat_np, columns = encoder.get_feature_names_out())
X_test_cat = pd.DataFrame(X_test_cat_np, columns = encoder.get_feature_names_out())


X_train_cat.head()

,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0


In [38]:
# applying logistic regression

In [44]:
X_train_transformed = np.concatenate([X_train_scaled, X_train_cat], axis=1)
log_reg= LogisticRegression(random_state=0, solver='saga').fit(X_train_transformed, y_train)

In [45]:
# concatining and log
X_test_transformed = np.concatenate([X_test_scaled, X_test_cat], axis=1)

In [43]:
predictions = log_reg.predict(X_test_transformed)

8. Evaluate the results.

In [49]:
# Model validation
log_reg.score(X_test_transformed,y_test)

0.9944444444444445